<a href="https://colab.research.google.com/github/OpenCodeEra/ML-X/blob/main/Email%20Spam%20Classification/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>